In [7]:
import matplotlib.pyplot as plt

from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read_events
from obspy import read_inventory
import cartopy.crs as ccrs

import pickle


In [2]:
catINGV=read_events('flegrei_2014_2023_INGV/catalogue_flegrei.xml')
catOV=read_events('flegrei_2014_2023_GOSSIP/catalogue_flegrei.xml')

In [4]:
id_catalogues=[]

for ind,ev in enumerate(catINGV):
    
    origin_time = ev.origins[0].time
    time_min= 'time > ' + str(origin_time - 5)
    time_max= 'time < ' + str(origin_time + 5)
    cat= catOV.filter(time_min , time_max)

    id_catalogues.append([])
    id_catalogues[ind].append(str(ev.resource_id)[-8:])
    #id_catalogues[ind].append(int(str(ev.resource_id)[-8:]))


    for match in cat:
        id_catalogues[ind].append(str(match.resource_id)[-8:])
        #id_catalogues[ind].append(int(str(match.resource_id)[-8:]))


id_catalogues

[['36846841', 'f5b7d629'],
 ['36833201', 'd9456ee8', '80bfcad3'],
 ['36794721', 'ab4e7480'],
 ['36758731', 'a5562e16'],
 ['36755891', '1c72697d'],
 ['36748281', '9c92c407'],
 ['36701861', 'bed62b9b'],
 ['36701841', '624c3043'],
 ['36697331', '87745093'],
 ['36692381', '0576c6dd'],
 ['36624321', '98e1ef98'],
 ['36614161', 'e7afa732'],
 ['36613631', '67519079'],
 ['36609981', '8603a736'],
 ['36605511', 'cd88aeda'],
 ['36603661', '9fa71d46'],
 ['36565471', '17b3c333'],
 ['36560281', '76434a52'],
 ['36559121', 'd78df4ea'],
 ['36529181', '764b5983'],
 ['36520811', '3e89d199'],
 ['36520801', 'ebe31f70', '507fea4c'],
 ['36521131', 'd9e598f0'],
 ['36520991', 'd52086b3'],
 ['36520981', '84848898'],
 ['36520771', '18cecf10'],
 ['36520871', '856faed1'],
 ['36520821', '1c1423f4'],
 ['36520181', '78b6d1c3'],
 ['36510951', '14706bb4'],
 ['36499161', '038c9289'],
 ['36495931', '7559bb3e'],
 ['36493851', '1e904d25'],
 ['36484541', '9954f55c'],
 ['36463761', 'e35e9845'],
 ['36452601', '1ee7171a'],
 ['3

In [6]:
print( 'len INGV:', len(catINGV), 'len comparison:', len(id_catalogues) )

len INGV: 741 len comparison: 741


## save

In [8]:
with open("event_comparison_INGV_GOSSIP", "wb") as fp:   #Pickling
    pickle.dump(id_catalogues, fp)

## load

In [10]:
with open("event_comparison_INGV_GOSSIP", "rb") as fp:   # Unpickling
    catalogues = pickle.load(fp)

In [11]:
catalogues

[['36846841', 'f5b7d629'],
 ['36833201', 'd9456ee8', '80bfcad3'],
 ['36794721', 'ab4e7480'],
 ['36758731', 'a5562e16'],
 ['36755891', '1c72697d'],
 ['36748281', '9c92c407'],
 ['36701861', 'bed62b9b'],
 ['36701841', '624c3043'],
 ['36697331', '87745093'],
 ['36692381', '0576c6dd'],
 ['36624321', '98e1ef98'],
 ['36614161', 'e7afa732'],
 ['36613631', '67519079'],
 ['36609981', '8603a736'],
 ['36605511', 'cd88aeda'],
 ['36603661', '9fa71d46'],
 ['36565471', '17b3c333'],
 ['36560281', '76434a52'],
 ['36559121', 'd78df4ea'],
 ['36529181', '764b5983'],
 ['36520811', '3e89d199'],
 ['36520801', 'ebe31f70', '507fea4c'],
 ['36521131', 'd9e598f0'],
 ['36520991', 'd52086b3'],
 ['36520981', '84848898'],
 ['36520771', '18cecf10'],
 ['36520871', '856faed1'],
 ['36520821', '1c1423f4'],
 ['36520181', '78b6d1c3'],
 ['36510951', '14706bb4'],
 ['36499161', '038c9289'],
 ['36495931', '7559bb3e'],
 ['36493851', '1e904d25'],
 ['36484541', '9954f55c'],
 ['36463761', 'e35e9845'],
 ['36452601', '1ee7171a'],
 ['3

In [13]:
ev_not_found=[]
ev_multiple_det=[]

for ev in catalogues:
    if len(ev)   >= 3:
        ev_multiple_det.append(ev)
    elif len(ev) == 1:
        ev_not_found.append(ev)
    elif len(ev) != 2:
        print('!event not categorized!')
        

2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
3
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
